In [1]:
from pandas import read_csv
from matplotlib import pyplot
import sys
import numpy as np
import pandas as pd

N=9
nowcast=1

#file="solartech_segmented.csv"
#file="univer_segmented.csv"
file="india_segmented.csv"

series=read_csv(file, sep=",", header=0,parse_dates=True)
series=series.values
series


array([[0, '2020-05-15 00:00:00', 0.0, 25.18431613333333, 0.0],
       [1, '2020-05-15 00:15:00', 0.0, 25.08458866666667, 0.0],
       [2, '2020-05-15 00:30:00', 0.0, 24.935752600000004, 0.0],
       ...,
       [3261, '2020-06-17 23:15:00', 0.0, 22.008274642857145, 0.0],
       [3262, '2020-06-17 23:30:00', 0.0, 21.96949473333333, 0.0],
       [3263, '2020-06-17 23:45:00', 0.0, 21.90928766666667, 0.0]],
      dtype=object)

In [2]:

X1=sensors=np.array(series[:,2])
X2=sensors=np.array(series[:,3])
Y0=sensors=np.array(series[:,4])

print(X1[0])
print(X2[0])
print(Y0[0])

0.0
25.18431613333333
0.0


In [3]:
import tensorflow as tf 

X1n=tf.keras.utils.normalize(    X1, axis=-1, order=2)
X1n=np.array(X1n[0])
#X1n=X1


print(X1n)
print(X1n.shape)

X2n=tf.keras.utils.normalize(X2, axis=-1, order=2)
X2n=np.array(X2n[0])
#X2n=X2
print(X2n)
print(X2n.shape)

X0=np.zeros((len(X1n),2))

X0[:,0]=X1n
X0[:,1]=X2n

Y=np.zeros((len(Y0),1))
Y[:,0]=Y0

print(X0)
print(X0.shape)
print(Y.shape)
print(Y)

[0.0 0.0 0.0 ... 0.0 0.0 0.0]
(3264,)
[0.0171390232809216 0.017071154399197833 0.01696986497771283 ...
 0.014977588808837745 0.014951197392503793 0.01491022386311301]
(3264,)
[[0.         0.01713902]
 [0.         0.01707115]
 [0.         0.01696986]
 ...
 [0.         0.01497759]
 [0.         0.0149512 ]
 [0.         0.01491022]]
(3264, 2)
(3264, 1)
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [4]:
X=[]
for i in range(N,len(X0)):
    X.append(X0[i-N:i])
X=np.array(X)    
print(X.shape)

Y=Y[N-nowcast:len(Y)-nowcast]
print(Y.shape)



from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler()
print(Y[40])
scaler.fit(Y)

Y = scaler.transform(Y)
print(Y[40])

(3255, 9, 2)
(3255, 1)
[7931.5]
[0.55415764]


In [5]:
print(X[0])
print(Y[0])

[[0.         0.01713902]
 [0.         0.01707115]
 [0.         0.01696986]
 [0.         0.01690887]
 [0.         0.01675602]
 [0.         0.01669788]
 [0.         0.01676769]
 [0.         0.01692717]
 [0.         0.01697067]]
[0.]


In [6]:
from sklearn.model_selection import train_test_split
x_train , x_val , y_train , y_val = train_test_split(X , Y ,test_size=0.2, random_state=0)

In [7]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D
from tensorflow.keras.models import Model

model = keras.Sequential()

model.add(Conv1D(filters=16, kernel_size=2, activation='relu', input_shape=(N,2)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    
model.add(Dropout(0.5))
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))

model.add(Flatten())

model.add(Dense(256))
model.add(Dense(128))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 8, 16)             80        
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 32)             1568      
                                                                 
 conv1d_2 (Conv1D)           (None, 4, 64)             6208      
                                                                 
 dropout (Dropout)           (None, 4, 64)             0         
                                                                 
 lstm (LSTM)                 (None, 4, 32)             12416     
                                                                 
 dropout_1 (Dropout)         (None, 4, 32)             0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                8

In [8]:
print(x_train.shape)
print(y_train.shape)




import tensorflow as tf
tf.get_logger().setLevel('ERROR')


def generate_true_samples(batch_size):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    x, true_labels = x_train[idx], y_train[idx]
    return x,true_labels
    
n_epochs=80
batch_size=64

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))


filew = open(file+'metrics_cgan_baseline.csv', 'w')

for epoch in range(n_epochs):
    print("e:",epoch)
    for batch_idx in range(int(x_train.shape[0] // batch_size)):
        
        
        x,true_labels = generate_true_samples(batch_size)
        model.train_on_batch(x,true_labels)

    print(epoch, " of ", n_epochs)
    
    yhat = model.predict(x_val)
   
    yhat0=scaler.inverse_transform(yhat).flatten()
    y_val0=scaler.inverse_transform(y_val).flatten()

    error0=tf.keras.metrics.mean_squared_error(
        y_val0, yhat0
    )
    print(np.sqrt(error0))    
    filew.write(str(np.sqrt(error0))+'\t')
    
    error=tf.keras.metrics.mean_squared_error(
        y_val.flatten(), yhat.flatten()
    )
    print(np.sqrt(error))    
    filew.write(str(np.sqrt(error))+'\t')
    
    yhat = model.predict(x_train)
   
    yhat0=scaler.inverse_transform(yhat).flatten()
    y_val0=scaler.inverse_transform(y_train).flatten()

    error0=tf.keras.metrics.mean_squared_error(
        y_val0, yhat0
    )
    print(np.sqrt(error0))    
    filew.write(str(np.sqrt(error0))+'\t')
    
    error=tf.keras.metrics.mean_squared_error(
        y_train.flatten(), yhat.flatten()
    )
    print(np.sqrt(error))    
    filew.write(str(np.sqrt(error))+'\n')    
filew.close()        
        
    #noise = getNoise()
    #x,true_labels = generate_true_samples(batch_size)
    #fake_labels = generator.predict([x,noise])
    #print("true_labels:",true_labels)
    #print("fake:",fake_labels)
    
    #print("g_loss:",g_loss)
    #print("d_loss:",d_loss)

(2604, 9, 2)
(2604, 1)
e: 0
0  of  80
21/21 [==============================] - 0s 2ms/step
3872.7717
0.27058262
82/82 [==============================] - 0s 2ms/step
3891.4343
0.27188656
e: 1
1  of  80
21/21 [==============================] - 0s 2ms/step
2102.7783
0.1469168
82/82 [==============================] - 0s 2ms/step
2084.5088
0.14564034
e: 2
2  of  80
21/21 [==============================] - 0s 2ms/step
1806.0167
0.12618269
82/82 [==============================] - 0s 2ms/step
1762.4064
0.123135716
e: 3
3  of  80
21/21 [==============================] - 0s 2ms/step
1671.3787
0.11677582
82/82 [==============================] - 0s 2ms/step
1633.801
0.11415033
e: 4
4  of  80
21/21 [==============================] - 0s 2ms/step
1612.1616
0.11263842
82/82 [==============================] - 0s 2ms/step
1566.4795
0.10944672
e: 5
5  of  80
21/21 [==============================] - 0s 2ms/step
1565.9962
0.10941295
82/82 [==============================] - 0s 2ms/step
1535.0538
0.107251056

82/82 [==============================] - 0s 2ms/step
377.9463
0.026406333
e: 50
50  of  80
21/21 [==============================] - 0s 2ms/step
376.64594
0.02631548
82/82 [==============================] - 0s 3ms/step
359.3846
0.025109466
e: 51
51  of  80
21/21 [==============================] - 0s 2ms/step
361.93277
0.025287502
82/82 [==============================] - 0s 2ms/step
342.91605
0.023958841
e: 52
52  of  80
21/21 [==============================] - 0s 3ms/step
368.8951
0.025773942
82/82 [==============================] - 0s 2ms/step
343.96408
0.024032068
e: 53
53  of  80
21/21 [==============================] - 0s 2ms/step
492.13278
0.034384307
82/82 [==============================] - 0s 2ms/step
489.94873
0.03423171
e: 54
54  of  80
21/21 [==============================] - 0s 2ms/step
454.50278
0.03175518
82/82 [==============================] - 0s 2ms/step
446.49557
0.031195726
e: 55
55  of  80
21/21 [==============================] - 0s 2ms/step
413.48203
0.02888914
82/82

In [9]:
from sklearn.metrics import confusion_matrix
yhat = model.predict(x_val)
yhat=scaler.inverse_transform(yhat).flatten()
y_val0=scaler.inverse_transform(y_val).flatten()
print(y_val)
print(yhat)

filew = open(file+'prediction.csv', 'w')
filew.write('ground truth\tprediction\n')
for i in range(0,len(y_val)):
    filew.write(str(y_val0[i])+"\t"+str(yhat[i])+"\n")
filew.close()

print("RMSE")
error=tf.keras.metrics.mean_squared_error(
    y_val0, yhat
)
print(np.sqrt(error))

21/21 [==============================] - 0s 2ms/step
[[0.12319716]
 [0.        ]
 [0.7950773 ]
 [0.03478114]
 [0.        ]
 [0.59180773]
 [0.04787701]
 [0.43096298]
 [0.50137241]
 [0.02525227]
 [0.26824302]
 [0.31014383]
 [0.70623522]
 [0.11740727]
 [0.        ]
 [0.        ]
 [0.53074689]
 [0.        ]
 [0.27218557]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.49968684]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.62212419]
 [0.        ]
 [0.84164928]
 [0.30818753]
 [0.17668731]
 [0.        ]
 [0.        ]
 [0.14887987]
 [0.19144692]
 [0.16275739]
 [0.00465121]
 [0.50203615]
 [0.90739502]
 [0.        ]
 [0.        ]
 [0.83473236]
 [0.72098733]
 [0.        ]
 [0.        ]
 [0.60956792]
 [0.        ]
 [0.2833582 ]
 [0.45229017]
 [0.        ]
 [0.00798241]
 [0.4073002 ]
 [0.        ]
 [0.22018011]
 [0.02149604]
 [0.35231413]
 [0.03152043]
 [0.17011099]
 [0.75296689]
 [0.09574529]
 [0.        ]
 [0.        ]
 [0.13531426]
 [0.        ]
 [0.        ]
 [0.13641